# Установка

In [ ]:
# @title Установка пакетов
from IPython.display import clear_output

!pip install --upgrade tiktoken
!pip install langchain openai
!pip install faiss-cpu

clear_output()

import getpass
import openai
import os
def get_key_ОpenAI():
  openai.api_key = getpass.getpass(prompt='Введите секретный ключ для сервиса chatGPT: ')
  os.environ["OPENAI_API_KEY"] = openai.api_key

get_key_ОpenAI()

# Для работы с Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Введите секретный ключ для сервиса chatGPT: ··········
Mounted at /content/drive


In [ ]:
#@title Импорт библиотек и Сервисные функции
import gdown
from IPython.display import clear_output
from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile

import time
import os
import openai
import tiktoken
import re
import os
import json
from tqdm.auto import tqdm
import getpass
from tqdm.auto import tqdm

# ----------------------------------
MODEL_TURBO_16K = "gpt-3.5-turbo-16k"
MODEL_TURBO_0613 = "gpt-3.5-turbo-0613"
MODEL_GPT4 = "gpt-4-0613"
# ----------------------------------

clear_output()

class WorkerОpenAI():
  def __init__(self, \
               system_promt = " ", \
               system_promt_lector = " ", \
               mod = MODEL_TURBO_16K, \
               content_topics = None, \
               save_project = '/content/'):
    self.model = mod
    self.save_project  = save_project

    if content_topics:
      self.content_topics = self.load_txt_file(content_topics)

    # системные настройки
    self.system_promt = self.load_document_text(system_promt)
    self.speaker_system_promt = self.load_document_text(system_promt_lector)


  def load_document_text(self, url: str) -> str:
      # функция для загрузки документа по ссылке из гугл док
      match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
      if match_ is None:
          raise ValueError('Invalid Google Docs URL')
      doc_id = match_.group(1)
      response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
      response.raise_for_status()
      text = response.text
      return text


  def load_txt_file(self, file_path):
      with open(file_path, 'r') as file_:
          text = file_.read()
      return text

  # пример подсчета токенов
  def num_tokens_from_messages(self, messages):
      """Returns the number of tokens used by a list of messages."""
      try:
          encoding = tiktoken.encoding_for_model(self.model)
      except KeyError:
          encoding = tiktoken.get_encoding("cl100k_base")
      # if self.model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
      if self.model in ["gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k", "gpt-4-0613"]:  # note: future models may deviate from this
          num_tokens = 0
          for message in messages:
              num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
              for key, value in message.items():
                  num_tokens += len(encoding.encode(value))
                  if key == "name":  # if there's a name, the role is omitted
                      num_tokens += -1  # role is always required and always 1 token
          num_tokens += 2  # every reply is primed with <im_start>assistant
          return num_tokens
      else:
          raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {self.model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")


  def create_embedding_faiss_db(self, doc_txt_dir="/content/", \
                                faiss_db_dir ="/content/", \
                                start_idx = 0, \
                                collection_name = " "):

    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    # Для Копирайтера
    self.splitter = RecursiveCharacterTextSplitter(['\n\n', '\n', ' '], chunk_size=1024, chunk_overlap=300)
    idx_file_folder = start_idx-1
    chunkID = idx_file_folder

    count_tokens = 0
    # проходимся по всем данным
    for _, file_ in enumerate(sorted(os.listdir(doc_txt_dir))):
        print("Загружается файл: ", file_)
        self.file_name = file_
        idx_file_folder +=1
        source_chunks = []
        # разбиваем на несколько частей с помощью метода split_text
        with open(doc_txt_dir + file_, "r") as f:
          for chunk in self.splitter.split_text(f.read()):
              chunkID += 1
              source_chunks.append(Document(page_content=chunk, \
                                  metadata={'source': file_,
                                            'chunkID': chunkID,
                                            "collection_name": collection_name,
                                            'idx_file_folder': idx_file_folder}))


        # Создание индексов документа и СОХРАНЕНИЕ
        # Если документ не пуст, то создать и сохранить базу индексов эмбеддингов отрезков документа
        if len(source_chunks) > 0:
            self.db = FAISS.from_documents(source_chunks, OpenAIEmbeddings())
            count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
            count_tokens += count_token
            print('Количество токенов в документе :', count_token)
            # print('ЦЕНА запроса:', 0.0004 * (count_token / 1000), ' $')

            self.db.save_local(os.path.join(faiss_db_dir, collection_name, f"{str(idx_file_folder)}_db_initial__{file_[:20]}"))

    print('\nЦЕНА запроса создания базы индексов:', 0.0004 * (count_tokens / 1000), ' $')

# ЗАПРС в ChatGPT
  def get_ChatCompletion(self, model,  # указываем модель
                         messages,     # словарь запроса
                         temp=0.1):    # температуру

      completion = openai.ChatCompletion.create(
        model= model,
        messages= messages,
        temperature= temp
        )

      print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
      # print('ЦЕНА запроса с ответом :', 0.0015*(completion["usage"]["total_tokens"]/1000), ' $')
      print('===========================================: \n')
      return completion.choices[0].message.content

# ЗАПРС на создание темы и подтемы по Материалам
  def get_search_materials_topics_subtopics(self, materials, model_topics = "gpt-3.5-turbo-16k"):

    messages = [
        {"role": "system", "content": f"{self.system_promt}"},
        {"role": "user", "content": f"""Проанализируй отрывок материала c семинара: {materials}.

Дай компактный, сжатый, обобщенный список тем и подтем.
Темы Необходимо оформить _#, а подтемы оформить ##_.
Используй только такой пример, ничего не добавляй лишнего.
Пример составления списка:
_#...
##_...
_#...
##_...
        """}
        ]

    # example token count from the function defined above
    print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.content_topics += self.get_ChatCompletion(model_topics, messages)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")


# ПОИСК ТЕМЫ и ПОДТЕМЫ
  def search_topics_subtopics(self, num_chunk = 10):
    self.content_topics = ''
    materials = ""
    # Выбираем блоки ПОДРЯД
    len_chunk = len(self.db.docstore._dict)
    if len_chunk < num_chunk + 1:
        for _, doc in self.db.docstore._dict.items():
            materials += f"{doc.page_content}\n"
        self.get_search_materials_topics_subtopics(materials)
    else:
        for ind, (key, doc) in enumerate(self.db.docstore._dict.items()):
          materials += f"{doc.page_content}\n"
          if (ind+1) % num_chunk == 0:
            self.get_search_materials_topics_subtopics(materials)
            materials = ""
    if materials != "":
        self.get_search_materials_topics_subtopics(materials)

    print('Собрали список тем и подтем: ')
    print(self.content_topics)

    with open(f'{self.save_project}{self.file_name[:20]}__темы_подтемы.txt', "w") as f:
      f.write(self.content_topics)


# ОБЪЕДИНЯЕМ схожие ТЕМЫ
  def get_merge_topics(self, model_topics = "gpt-3.5-turbo-16k"):

    messages = [
        {"role": "system", "content": f"{self.system_promt}"},
        {"role": "user", "content": f"""Проанализируй Темы и подтемы Семинара: {self.content_topics}.

Необходимо объединить похожие по смыслу темы или подтемы, записать компактно.
При необходимости перефразировать тему или подтему. Дай корректный список.
Темы оформи _#, а подтемы оформи ##_.
Используй только такой пример, ничего не добавляй лишнего.
Пример составления списка:
_#...
##_...
_#...
##_...
"""}
]
    # example token count from the function defined above
    print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.content_topics = self.get_ChatCompletion(model_topics, messages)
      print('Итоговый список тем и подтем: ')
      print(self.content_topics)
      with open(f'{self.save_project}{self.file_name[:20]}__темы_подтемы_ИТОГ.txt', "w") as f:
        f.write(self.content_topics)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")

# ОРГАНИЗУЕМ текст блоками
  def organize_text(self, topic, num_chunks, model_topics):

    # Выборка документов по схожести с подтемой
    docs = self.db.similarity_search(topic, k = num_chunks)
    message_content = '\n'.join([doc.page_content + '\n' for i, doc in enumerate(docs)])

    messages = [
        {"role": "system", "content": f"{self.speaker_system_promt}"},
        {"role": "user", "content": f"""Вот отрывки Вашей лекции:\n{message_content}

Опираясь только на информацию с Лекции, указанной выше, расскажите подробнее по Теме: {topic}.
"""}
      ]

    # example token count from the function defined above
    print('\n ===========================================: ')
    print(f"{self.num_tokens_from_messages(messages=messages)} токенов использовано на вопрос \n")
    try:
      self.final_text += self.get_ChatCompletion(model_topics, messages)
    except:
      print("Модель в настоящее время перегружена. Попробуйте позже.")

# ОРГАНИЗУЕМ ИТОГОВЫЙ ТЕКСТ
  def organize_final_text(self, num_chunks = 10,
                          model_topics = "gpt-3.5-turbo-16k",
                          name = None,
                          db_path = None):
    if name:
      self.file_name = name

    if db_path:
      for curr_base in os.listdir(db_path):
          self.db = FAISS.load_local(os.path.join(db_path, curr_base), OpenAIEmbeddings())

    self.final_text = ""
    list_topics = self.content_topics.split('\n')
    # проходимся по списку тем и подтем
    for ind, topic in tqdm(enumerate(list_topics)):
        # тему просто записываем в итоговый текст
        if ("_#" in topic) and not("_#" in list_topics[ind+1]):
          self.final_text += f'\n{topic}\n'
        # подтему передаем в ChatGPT
        else:
          self.final_text += f'\n{topic}\n'
          self.organize_text(topic, num_chunks, model_topics)

    with open(f'{self.save_project}{self.file_name[:20]}__final_text.txt', "w") as f:
      f.write(self.final_text)
    print("\nСоздали финальный документ для Базы Знаний (final_text)")





clear_output()




# Copywriter

In [ ]:
#@title Создаем объект для работы Copywriter
projects_dir = '/content/drive/MyDrive/Colab Notebooks/_Projects_ChatGPT/Нейро_Copywriter/УИИ/'

Promt_copywriter = "https://docs.google.com/document/d/1Pdmys-RHFa7c_teMv8AJkgOfFhK0sHij285CycfKGvs/edit?usp=sharing"
Promt_lector = "https://docs.google.com/document/d/1HYA19tp2BP08IL5ltHYjssEen70rIS-jy44E6hmt-Jo/edit?usp=sharing"

# если темы созданы
# topics_final = "/content/drive/MyDrive/Colab Notebooks/_Projects_ChatGPT/Нейро_Copywriter/УИИ/___.txt"
# # Создаем объект для дообучения chatGPT
curator = WorkerОpenAI(system_promt = Promt_copywriter, # системный промт
                       system_promt_lector = Promt_lector, # промт Лектора
                      #  content_topics = topics_final,
                       save_project = projects_dir)     # путь для сохранения готовых файлов

# Формируем базу по файлу txt
# путь к материалам
doc_txt_dir = projects_dir + 'BASE/'
db_initial = projects_dir

curator.create_embedding_faiss_db(doc_txt_dir = doc_txt_dir,   # путь к материалам
                                faiss_db_dir = db_initial,     # путь для сохранения исходной базы
                                start_idx = 0,                 # номер документа в базе
                                collection_name = 'УИИ_db_initial')  # наименование коллекции

Загружается файл:  Разбор нейро-сотрудников на chatGPT _ 21.08.2023_summ.txt
Количество токенов в документе : 18352

ЦЕНА запроса создания базы индексов: 0.007340800000000001  $


In [ ]:
# Смотрим созданные чанки
print(f"Текст разбит на чанки. Всего: {len(curator.db.docstore._dict.values())} шт.\n")
curator.db.docstore._dict.values()

Текст разбит на чанки. Всего: 56 шт.



dict_values([Document(page_content='Добрый день, уважаемые участники вебинара! Сегодня я хотел бы поделиться с вами информацией о ChatGPT и нейросотрудниках. Но перед этим хочу сообщить, что уже через 4 дня стартует курс ChatGPT Professional, и мы очень ждем всех желающих присоединиться.\nНа прошлой неделе у нас был вебинар, на котором было много интересных выступлений от наших участников. У нас действительно потрясающая команда, и я очень горжусь тем, что работаю с ними. Недавно мы отметили 5-летие нашей компании, и я понял, что это именно то, о чем я всегда мечтал.\nТеперь перейдем к курсу ChatGPT Professional. Ведущими курса будут Света и Миша. Света уже была ведущей мини-курса, а Миша отлично объясняет материал тем, кто только начинает. К сожалению, Серега покидает нашу команду, но мы уже выбираем замену для его части по интеграции и продажам.', metadata={'source': 'Разбор нейро-сотрудников на chatGPT _ 21.08.2023_summ.txt', 'chunkID': 0, 'collection_name': 'УИИ_db_initial', 'idx_

In [ ]:
#@title Поиск темы и подтемы выборкой по 5 - 15 чанков

# Подаем в цикле и просим составить список тем и подтем.
curator.search_topics_subtopics(num_chunk = 15)

5330 токенов использовано на вопрос 

5962 токенов использовано всего (вопрос-ответ).
===========================================: 

4882 токенов использовано на вопрос 

5048 токенов использовано всего (вопрос-ответ).
===========================================: 

5410 токенов использовано на вопрос 

5973 токенов использовано всего (вопрос-ответ).
===========================================: 

3754 токенов использовано на вопрос 

4474 токенов использовано всего (вопрос-ответ).
===========================================: 

Собрали список тем и подтем: 
_#Семинар о ChatGPT и нейросотрудниках_
##_Старт курса ChatGPT Professional
##_Ведущие курса
##_Информация о нейросотрудниках ChatGPT
_#Искусственный интеллект и капча_
##_Процент прохождения капчи у искусственного интеллекта и людей
##_Конкуренция между людьми и искусственным интеллектом в рекапче
_#Новости в области чат-ботов и искусственного интеллекта_
##_Новые платформы Белуга и Утконос
##_Планы Google на выпуск систем Gemini
##_

In [ ]:
#@title Корректируем список тем и подтем (обобщаем, убираем дубли).
# Подаем сформированный Список тем и просим объединить похожие по смыслу темы или подтемы, записать компактно.
# При необходимости перефразировать тему или подтему.
curator.get_merge_topics()

2341 токенов использовано на вопрос 

4400 токенов использовано всего (вопрос-ответ).
===========================================: 

Итоговый список тем и подтем: 
_#Семинар о ChatGPT и нейросотрудниках_
##_Старт курса ChatGPT Professional
##_Ведущие курса
##_Информация о нейросотрудниках ChatGPT

_#Искусственный интеллект и капча_
##_Процент прохождения капчи у искусственного интеллекта и людей
##_Конкуренция между людьми и искусственным интеллектом в рекапче

_#Новости в области чат-ботов и искусственного интеллекта_
##_Новые платформы Белуга и Утконос
##_Планы Google на выпуск систем Gemini
##_Сотрудничество с компаниями, специализирующимися на чат-ботах
##_Применение чат-ботов в различных сферах

_#Создание нейросотрудников на базе ChatGPT_
##_Дообучение ChatGPT на базе знаний компании
##_Роль базы знаний и промпта в работе нейросотрудника
##_Процесс создания и тестирования нейроменеджера чата
##_Использование нейросотрудника в различных каналах коммуникации

_#Структурирование инф

## ОРГАНИЗУЕМ ИТОГОВЫЙ ТЕКСТ

Выборка блоков документов по схожести с подтемой. Передаем собранные отрывки с Лекции и просим рассказать подробнее, опираясь на Лекцию.

In [ ]:
# запускаем сбору Итогового Текста
# name_file = 'Разбор нейро-сотрудников на chatGPT _ 21.08.2023'
# db_initial = '/content/drive/MyDrive/Colab Notebooks/_Projects_ChatGPT/Нейро_Copywriter/УИИ/УИИ_db_initial'

# curator.organize_final_text(name = name_file, db_path = db_initial)
curator.organize_final_text()

0it [00:00, ?it/s]


 ===========================================: 
3723 токенов использовано на вопрос 

4922 токенов использовано всего (вопрос-ответ).
===========================================: 


 ===========================================: 
3697 токенов использовано на вопрос 

4236 токенов использовано всего (вопрос-ответ).
===========================================: 


 ===========================================: 
3803 токенов использовано на вопрос 

4898 токенов использовано всего (вопрос-ответ).
===========================================: 


 ===========================================: 
3260 токенов использовано на вопрос 

4481 токенов использовано всего (вопрос-ответ).
===========================================: 


 ===========================================: 
3892 токенов использовано на вопрос 

4758 токенов использовано всего (вопрос-ответ).
===========================================: 


 ===========================================: 
3617 токенов использовано на вопрос 

4576 токе

In [ ]:
# сформирован итоговый текст
curator.final_text

'\n_#Семинар о ChatGPT и нейросотрудниках_\n\n##_Старт курса ChatGPT Professional\nСтарт курса ChatGPT Professional - это событие, которое мы с нетерпением ждем уже через 4 дня. Этот курс предназначен для всех, кто хочет стать профессионалом в области нейросотрудников и ChatGPT. Ведущими курса будут Света и Миша, которые уже имеют опыт в преподавании и объяснении материала. Курс будет длиться 8 месяцев и будет включать в себя различные модули и темы, связанные с ChatGPT и его применением.\n\nОдной из ключевых целей курса является обучение участников работе с нейросотрудниками. На лекции было упомянуто, что нейросотрудники - это новая золотая жила, и они станут неотъемлемой частью нашего будущего. Курс ChatGPT Professional предоставит участникам все необходимые знания и навыки для работы с нейросотрудниками и их интеграции в различные сферы деятельности.\n\nОдин из аспектов, который будет рассмотрен на курсе, - это обзор различных нейросотрудников ChatGPT. Лектор обещает рассказать о 9 

# MarkdownHeaderTextSplitter

In [ ]:
#@title функции
chat_manager_system = """
Ты программист, специалист по Нейронным сетям, Профессионал Data Science и Преподаватель в Университете Искусственного Интеллекта.
Ты Спикер на Вебинаре.
Тебе предоставят отрывок с Вебинара и попросят раскрыть подробнее одну из тем.

Твоя цель: Опираясь только на материал с Вебинара, подробно, развернуто рассказать по интересующей теме на русском языке.
"""

def get_chatgpt_answer(topic,  db, model = MODEL_TURBO_0613):
  # Выборка документов по схожести с вопросом
  docs = db.similarity_search(topic, k=4)
  message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\n  ' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
  # print('message_content :\n ======================================== \n', self.message_content)

  messages = [
    {"role": "system", "content": f"{chat_manager_system}"},
    {"role": "user", "content": f"""Analyze step by step and give a detailed correct answer to the Student's question.\n
    Question:\n{topic}\n\nMaterials from the Webinar:\n{message_content}\n\nAnswer:"""}
    ]

  try:
    completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=0.1
    )

    print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    print('ЦЕНА запроса с ответом :', 0.0015*(completion["usage"]["total_tokens"]/1000), ' $')
    print('===========================================: \n')
    print('Ответ ChatGPT: ')
    print(completion.choices[0].message.content)
    # return completion.choices[0].message.content
  except:
    print("Модель в настоящее время перегружена. Попробуйте позже.")

def load_txt_file(file_path):
    with open(file_path, 'r') as file_:
        text = file_.read()
    return text

projects_dir = '/content/drive/MyDrive/Colab Notebooks/_Projects_ChatGPT/Нейро_Copywriter/УИИ/'
name_file = '15_Разбор нейро-сотруд__final_text.txt'
final_text_dir = projects_dir + name_file

# Загружаем итоговый текст
final_text = load_txt_file(final_text_dir)


In [ ]:
#@title  Готовим документ MarkdownHeader по сформированному тексту
from langchain.text_splitter import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("_#", "Header 1"),
    ("##_", "Header 2"),
    ("###_", "Header 3"),
    ("####_", "Header 4"),
    ("#####_", "Header 5"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(final_text)
md_header_splits

[Document(page_content='_#Семинар о ChatGPT и нейросотрудниках_  \n##_Старт курса ChatGPT Professional\nСтарт курса ChatGPT Professional - это событие, которое мы с нетерпением ждем уже через 4 дня. Этот курс предназначен для всех, кто хочет стать профессионалом в области нейросотрудников и ChatGPT. Ведущими курса будут Света и Миша, которые уже имеют опыт в преподавании и объяснении материала. Курс будет длиться 8 месяцев и будет включать в себя различные модули и темы, связанные с ChatGPT и его применением.  \nОдной из ключевых целей курса является обучение участников работе с нейросотрудниками. На лекции было упомянуто, что нейросотрудники - это новая золотая жила, и они станут неотъемлемой частью нашего будущего. Курс ChatGPT Professional предоставит участникам все необходимые знания и навыки для работы с нейросотрудниками и их интеграции в различные сферы деятельности.  \nОдин из аспектов, который будет рассмотрен на курсе, - это обзор различных нейросотрудников ChatGPT. Лектор об

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024, chunk_overlap=300
)

# Split
split = text_splitter.split_documents(md_header_splits)
db = FAISS.from_documents(split, OpenAIEmbeddings())

In [ ]:
split[1]

Document(page_content='Один из аспектов, который будет рассмотрен на курсе, - это обзор различных нейросотрудников ChatGPT. Лектор обещает рассказать о 9 различных нейросотрудниках и показать простые и типовые слайды про каждого из них. Это позволит участникам получить представление о возможностях и особенностях каждого нейросотрудника и выбрать наиболее подходящего для своих задач.  \nКроме того, на курсе будут рассмотрены и другие важные аспекты работы с ChatGPT. Будет изучаться копирайтинг и работа с базами данных, что позволит эффективно тестировать нейросотрудников. Также будет рассмотрена работа с промптами и базой знаний компании, которые играют важную роль в обучении нейросотрудника и предоставлении точных и специфических ответов на вопросы пользователей.', metadata={})

In [ ]:
topic = """
Если я выберу тариф GPT Professional Лайт, что в него входит?
"""

get_chatgpt_answer(topic,  db)

2320 токенов использовано всего (вопрос-ответ).
ЦЕНА запроса с ответом : 0.00348  $
===========================================: 

Ответ ChatGPT: 
Тариф GPT Professional Лайт включает в себя следующие услуги и возможности. 

Во-первых, данный тариф предоставляет доступ к курсу "GPT Professional", который проводится вживую. Однако, для удобства студентов, некоторые части курса записываются и ускоряются, чтобы студенты могли пройти основную часть курса до Нового года. Таким образом, студенты, выбравшие тариф GPT Professional Лайт, смогут получить доступ к записям и просмотреть их позже, если у них нет возможности присутствовать на вебинаре вживую.

Второй важной особенностью тарифа GPT Professional Лайт является возможность пройти курс по часам. Это означает, что студенты могут использовать 20 часов Zoom консультаций с кураторами курса. Эти часы можно использовать как для консультаций, так и для рабочих часов разработчиков компании. Таким образом, студенты получают возможность получить д